<a href="https://colab.research.google.com/github/ajevnisek/Adversarial-Attacks-with-Relativistic-AdvGAN/blob/master/RadarExtendedProblemSet3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Set #3

## Exercise #1

### Exercise 1a:

The delay:

$\tau _0 = 2 \cdot \frac{\sqrt{x_0^2 + y_0^2 + h^2}}{c}$

The doppler:

$\nu _0 = \frac{V}{c} \cdot f \cdot \cos (\text{projection to radial-velocity angle}) = \frac{V}{c} \cdot \frac{c}{\lambda} \cdot \frac{x_0}{\sqrt{x_0^2 + y_0^2 + h^2}} = \frac{V}{\lambda} \cdot \frac{x_0}{\sqrt{x_0^2 + y_0^2 + h^2}}$

### Exercise 1b:


Target further from the radar than the reference $\tau = 0$ position corresponds to positive $\tau$.

Here the land is at $(x, y, z) = (2000, -2100, 0)$ and the radar moves towards positive $x$s which means it is moving towards the target (and therefore the target is moving towards the radar). Hence the $\tau$ is positive $\tau$.


A positive $\nu$ implies a target moving towards the radar. Here the radar approaches the target (moving towards positive $x$s) so the sign of this is also positive.



In [1]:
from math import sqrt
h = 3000 # [m]
V = 200 # [m/sec]
f = 10 * 10 ** 9 # [Hz]
x_0 = 2000 # [m]
y_0 = -2100 # [m]
c = 3 * 10 **8 # [m/sec]
lmbda = c / f

tau_0 = 2 * sqrt(x_0 **2  + y_0 **2 + h ** 2) / c
nu_0 = (V / lmbda) * (x_0 / sqrt(x_0 **2  + y_0 **2 + h ** 2))
print(f"tau_0 is: {tau_0 * 10 ** 6:2f} [micro-sec]")
print(f"nu_0 is: {nu_0 / 10 **3 :.2f} [KHz]")

tau_0 is: 27.816861 [micro-sec]
nu_0 is: 3.20 [KHz]


## Exercise #2

The LFM pulse is given by:

$u(t) = \frac{1}{\sqrt{t_p}}\cdot \text{rect}\Big ( \frac{t}{t_p} \Big ) \cdot\exp (j \cdot \pi \cdot k \cdot t^2)$

TBW = Time bandwidth = compression ratio = if we transmit a signal with duration $T_p$, then the autocorrelation resolution would be: $\frac{T_p}{TBW}$.

To calculate the autocorrelation function for this pulse, we will do the following:
1. Calculate the square pulse ambiguity function.
2. Calculate the square pulse modulated with linear frequecy according to property (4) (linear FM) shown in class.
3. Slice the ambiguity function in zero doppler (zero doppler cut) to get the autocorrelation function.

Let's do it:

1. Square pulse ambiguity function is:

$|\chi (\tau, \nu)| = \Big | \Big ( 1 - \frac{|\tau|}{t_p} \Big ) \frac{\sin \Big [ \pi t_p \nu \Big ( 1 - \frac{|\tau|}{t_p} \Big ) \Big ] }{ \pi t_p \nu \Big ( 1 - \frac{|\tau|}{t_p} \Big )} \Big | , |\tau| \leq t_p$

2. Square pulse modulated with linear frequecy: Substitute $\nu → \nu - k \tau$

$|\chi (\tau, \nu)| = \Big | \Big ( 1 - \frac{|\tau|}{t_p} \Big ) \frac{\sin \Big [ \pi t_p \cdot (\nu - k \tau) \cdot  \Big ( 1 - \frac{|\tau|}{t_p} \Big ) \Big ] }{ \pi t_p \cdot (\nu - k \tau) \cdot \Big ( 1 - \frac{|\tau|}{t_p} \Big )} \Big |, |\tau| \leq t_p$


3. Autocorrelation function: $\nu = 0$:

$ACF = |\chi (\tau, \nu=0)| = \Big | \Big ( 1 - \frac{|\tau|}{t_p} \Big ) \frac{\sin \Big [ \pi t_p k \tau \cdot  \Big ( 1 - \frac{|\tau|}{t_p} \Big ) \Big ] }{ \pi t_p \cdot k \tau \cdot \Big ( 1 - \frac{|\tau|}{t_p} \Big )} \Big |, |\tau| \leq t_p$


Note that the minuses cancelled in the sine-fraction.

Now let's plot it with numpy


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (6, 6)
L = 1000

tp = 1.0
k = 20.0
t = np.linspace(-tp / 2.0, tp / 2.0, L)
acf_theoretical = np.abs(
 (1 - t / tp) * 
 (np.sin(np.pi * tp * k * t * (1 - t / tp)) / (np.pi * tp * k * t * (1 - t / tp)))
)

plt.title('theoretical ACF:')
plt.plot(t, acf_theoretical)
plt.grid(True)
plt.xlabel('t / tp')


Let's simulate the pulse:

In [ ]:
u = 1 / np.sqrt(tp) * np.exp(1j * np.pi * k * ((t + tp / 2)** 2))

plt.rcParams['figure.figsize'] = (12, 6)
# our Linear-FM pulse
plt.subplot(1, 2, 1)
plt.title('real part')
plt.plot(t, np.real(u))
plt.xlabel('t / tp')
plt.subplot(1, 2, 2)
plt.title('imagionary part')
plt.xlabel('t / tp')
plt.plot(t, np.imag(u))




In [ ]:
acf = np.correlate(u, u, 'full')
plt.title('Auto Correlation Function')
plt.grid(True)
plt.plot(np.linspace(-tp , tp , L*2 - 1), np.abs(acf) / L )


In [ ]:
plt.title('Auto Correlation Function - zoom in first null')
plt.plot(np.linspace(-tp , tp , 2*L - 1), np.abs(acf) / np.max(np.abs(acf)))
eps = 1 / 30.0
plt.xlim([1 / 20.0 - eps, 1 / 20.0 + eps ])
plt.grid(True)

In [ ]:
plt.title('Auto Correlation Function - zoom in last null')
plt.plot(np.linspace(-tp , tp , 2*L - 1), np.abs(acf) / np.max(np.abs(acf)))
eps = 1 / 30.0
plt.xlim([1-1 / 20.0 - eps, 1-1 / 20.0 + eps ])
plt.ylim([0, 0.02])
plt.grid(True)

## Exercise 2b:
Zeros for the ACF function occur when the sine argument is at zero (except $tau=0$ in which case the ACF is maximal [maximal acutocorrelation when the signal is aligned with itself]).

$ACF = |\chi (\tau, \nu=0)| = \Big | \Big ( 1 - \frac{|\tau|}{t_p} \Big ) \frac{\sin \Big [ \pi t_p k \tau \cdot  \Big ( 1 - \frac{|\tau|}{t_p} \Big ) \Big ] }{ \pi t_p \cdot k \tau \cdot \Big ( 1 - \frac{|\tau|}{t_p} \Big )} \Big |, |\tau| \leq t_p$

We would like to show that the zeros are symmetrical around $\frac{\tau}{t_p} = 0.5$. To do that, we define a new variable: $\tau' = \tau - 0.5t_p$. Showing that the sine is symmetric around this new variable yields that the zeros are symmetric around $\tau'$:

$\text{arg}(sin) = \pi t_p k \tau \cdot  \Big ( 1 - \frac{|\tau|}{t_p} \Big ) = \pi t_p k \cdot (\tau' + 0.5 t_p) \cdot  \Big ( 1 - \frac{|\tau' + 0.5 t_p|}{t_p} \Big ) = \pi t_p k \cdot (\tau' + 0.5 t_p) \cdot \frac{0.5t_p - \tau'}{t_p} = $ 

$= \pi k \cdot (\tau' + 0.5 t_p) \cdot (0.5t_p - \tau')= \pi k \cdot (0.25 t_p^2 - \tau' ^2)$ 

The function: $f(\tau') = 0.25 t_p^2 - \tau' ^2$ is symmetrical around symmetric around $\tau' = 0$, which ends the proof.


## Exercise 2c:



In [ ]:
plt.rcParams['figure.figsize'] = (16, 16)
t = np.linspace(-tp / 2.0, tp / 2.0, L)
u = 1 / np.sqrt(tp) * np.exp(1j * np.pi * k * ((t + tp / 2)** 2))
afs = {}
for idx, nu in enumerate([0.1, 0.5, 1, 2]):
  u_doppler = u * np.exp(1j * np.pi * nu * t)
  af = np.correlate(u, u_doppler, 'full')
  af = np.abs(af) / len(np.abs(af))
  afs[nu] = af
  plt.subplot(2, 2, idx + 1)
  plt.title(f'Ambiguity Slice @ nu*t_p={nu}')
  plt.plot(np.linspace(-tp , tp , L*2 - 1), af)
  plt.grid(True)
  plt.xlabel('tau / t_p')


In [ ]:
import pandas as pd
from tabulate import tabulate
from collections import OrderedDict


afs_time = np.linspace(-tp , tp , L*2 - 1)
stats = {}

for nu in afs:
  af = afs[nu]
  pretty_nu = f'nu * tp = {nu}'
  stats[pretty_nu] = OrderedDict({})
  stats[pretty_nu]['max-height'] = max(af)
  stats[pretty_nu]['max-location'] = afs_time[np.argmax(af)]
  first_zero_pos_right = [pos for pos, x in enumerate(af) if x < 10**-2 and pos > np.argmax(af)][0]
  stats[pretty_nu]['first-zero-right'] = afs_time[first_zero_pos_right]
  first_zero_pos_left = [pos for pos, x in enumerate(af) if x < 10**-2 and pos < np.argmax(af)][-1]
  af_zeroed_main_lobe = [x if pos > first_zero_pos_right or pos < first_zero_pos_left else 0
                         for pos, x in enumerate(af)]
  # print(first_zero_pos_left, first_zero_pos_right)
  # plt.plot(af_zeroed_main_lobe)
  stats[pretty_nu]['side-lobe-height'] = max(af_zeroed_main_lobe)
  stats[pretty_nu]['side-lobe-location'] = afs_time[np.argmax(af_zeroed_main_lobe)]
df = pd.DataFrame(stats)
print(tabulate(df, headers = 'keys', tablefmt = 'psql'))



